In [5]:
import numpy as np
# import mahotas as mh
import os
# import matplotlib.pyplot as plt
import cv2
import pywt
import pywt.data

### 이미지 스택으로부터 EDOF 결합 함수 (Multi level Wavelet Transform)

In [2]:
def combine_focus_stack(stack, num_levels):
    # 초기화: 결과 이미지 생성 및 모든 픽셀을 0으로 설정
    result = np.zeros_like(stack[0], dtype=np.float32)
    max_coeff = np.zeros_like(result)

    # 스택 내의 모든 이미지에서 해당 위치의 픽셀 값 비교
    for k in range(len(stack)):
        # 다중 레벨 웨이블릿 변환 적용 (예시: 'haar')
        coeffs = pywt.wavedec2(stack[k], 'haar', level=num_levels)

        for level in range(1, num_levels + 1):
            # LL(저주파 영역) 세부 계수 추출 및 절대값 적용 (웨이블릿 세부 계수는 음수일 수 있음)
            LL_abs = np.abs(coeffs[level][0])

            for i in range(result.shape[0]):
                for j in range(result.shape[1]):
                    if LL_abs[i, j] > max_coeff[i, j]:
                        max_coeff[i, j] = LL_abs[i, j]
                        result[i, j] = stack[k][i, j]

    return result.astype(np.uint8)

### Image load

In [ ]:
image_folder = "/works/pythonProject/Test Image/10x_40deg_step20um"  # 이미지 폴더 경로 설정
output_folder = "/works/pythonProject/Test Image/10x_40deg_step20um/result"  # 결과 이미지 저장 폴더 경로 설정

# 결과 이미지 저장 폴더가 없으면 생성하기
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 이미지 폴더에서 파일 목록 가져오기
file_list = os.listdir(image_folder)
img_files = [file for file in file_list if file.endswith('.bmp')]
# image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

stack = []
for file_name in image_files:
    file_path = os.path.join(image_folder, file_name)
    
    # 각 파일을 그레이스케일로 읽어와서 스택에 추가하기
    img_gray = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    
    if img_gray is not None:
        stack.append(img_gray)

### EDOF Processing

In [ ]:
num_levels = 3  # 다중 레벨 수 설정

if len(stack) > 0:
    # 다중 초점 이미지 결합 수행
    result_image = combine_focus_stack(stack, num_levels)

    output_file_path = os.path.join(output_folder, "EDOF_image.jpg")
     # 결과 저장
     cv2.imwrite(output_file_path , result_image)
     print("Combined image saved successfully.")
else:
     print("No images found in the folder.")